In [1]:
from google.colab import files
uploaded = files.upload()

Saving credentials.csv to credentials.csv


# Authenticate

In [0]:
import boto3
import pandas as pd

CREDENTIALS_FILE = 'credentials.csv'
credentials = pd.read_csv(CREDENTIALS_FILE).to_dict('records')[0]
aws_access_key_id = credentials['Access key ID']
aws_secret_access_key = credentials['Secret access key']

region_name = 'us-east-1'
endpoint_url = 'https://mturk-requester-sandbox.us-east-1.amazonaws.com'

# Uncomment this line to use in production
# endpoint_url = 'https://mturk-requester.us-east-1.amazonaws.com'
 
client = boto3.client(
    'mturk',
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    endpoint_url=endpoint_url,
    region_name=region_name
)

#Get  Balance

In [3]:
print(client.get_account_balance())

{'AvailableBalance': '10000.00', 'ResponseMetadata': {'RequestId': '83a45cef-4fa9-4f63-a3ee-620045f0a46e', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '83a45cef-4fa9-4f63-a3ee-620045f0a46e', 'content-type': 'application/x-amz-json-1.1', 'content-length': '31', 'date': 'Sat, 23 May 2020 07:29:44 GMT'}, 'RetryAttempts': 0}}


#Create HIT Type

In [0]:
one_minute = 60 # seconds
one_hour = 60 * one_minute
one_day = 24 * one_hour

hit_type_response = client.create_hit_type(
    AutoApprovalDelayInSeconds=10 * one_minute,
    AssignmentDurationInSeconds=5 * one_minute,
    Reward='0.10',
    Title='Select a Color',
    Keywords='color,language,fun',
    Description='Choose the color that most matches its name',
    QualificationRequirements=[
        {
            'QualificationTypeId': '00000000000000000071', # see https://docs.aws.amazon.com/AWSMechTurk/latest/AWSMturkAPI/ApiReference_QualificationRequirementDataStructureArticle.html#ApiReference_QualificationType-IDs
            'Comparator': 'In',
            'LocaleValues': [
                {
                    'Country': 'TW'
                },
                {
                    'Country': 'JP'
                },
            ],
            'RequiredToPreview': True,
            'ActionsGuarded': 'PreviewAndAccept'
        },
    ]
)


In [0]:
hit_type_id = hit_type_response['HITTypeId']

In [0]:
question='''<?xml version="1.0" encoding="UTF-8"?>
<ExternalQuestion xmlns="http://mechanicalturk.amazonaws.com/AWSMechanicalTurkDataSchemas/2006-07-14/ExternalQuestion.xsd">
  <ExternalURL>https://bshmueli.github.io/108-nctu-mturk/pink.html</ExternalURL>
  <FrameHeight>800</FrameHeight>
</ExternalQuestion>'''

response = client.create_hit_with_hit_type(
    HITTypeId=hit_type_id,
    MaxAssignments=10,
    LifetimeInSeconds=1 * one_hour,
    Question=question,
    RequesterAnnotation='pink',
)

In [0]:
question='''<?xml version="1.0" encoding="UTF-8"?>
<ExternalQuestion xmlns="http://mechanicalturk.amazonaws.com/AWSMechanicalTurkDataSchemas/2006-07-14/ExternalQuestion.xsd">
  <ExternalURL>https://bshmueli.github.io/108-nctu-mturk/blue.html</ExternalURL>
  <FrameHeight>800</FrameHeight>
</ExternalQuestion>'''

response = client.create_hit_with_hit_type(
    HITTypeId=hit_type_id,
    MaxAssignments=10,
    LifetimeInSeconds=one_hour,
    Question=question,
    RequesterAnnotation='blue',
)

Fetch results

In [0]:
# list_hits_response = client.list_hits()
# hit_id = list_hits_response['HITs'][0]
# response = client.list_assignments_for_hit(HITId=hit_id)

In [0]:
hits_paginator = client.get_paginator('list_hits')
assignments_paginator = client.get_paginator('list_assignments_for_hit')

for hits in hits_paginator.paginate():
    for hit in hits['HITs']:
        for assignments in assignments_paginator.paginate(HITId=hit['HITId']):
            for assignment in assignments['Assignments']:
                print(assignment['Answer'])